In [2]:
import pandas as pd
from db_utils import Query, QueryAll, QueryAllFinished, UpdateItem

pd.options.mode.chained_assignment = None

def convert_currency(original):
    EUR_TO_GBP = 0.87
    USD_TO_GBP = 0.84
    AUD_TO_GBP = 0.56
    CAD_TO_GBP = 0.63
    original = original.replace(',','')
    if original.startswith('£'):
        return float(original[1:])
    elif original.startswith('EUR'):
        return float(original[4:])*EUR_TO_GBP
    elif original.startswith('US'):
        return float(original[4:])*USD_TO_GBP
    elif original.startswith('AU'):
        return float(original[4:])*AUD_TO_GBP
    elif original.startswith('C'):
        return float(original[3:])*CAD_TO_GBP
    
WEEKENDS = ['2022-10-01', '2022-10-02', '2022-10-07', '2022-10-08', '2022-10-09', '2022-10-14', 
            '2022-10-15', '2022-10-16', '2022-10-21', '2022-10-22', '2022-10-23', '2022-10-28', 
            '2022-10-29', '2022-10-30', '2022-11-04', '2022-11-05', '2022-11-06', '2022-11-11', 
            '2022-11-12', '2022-11-13', '2022-11-18', '2022-11-19', '2022-11-20','2022-11-25',
            '2022-11-26','2022-11-27']

USED = [3000, 6000, 2500, 5000, 2750]
NEW = ['1000', '1500']
BROKEN = [7000]

cols = ["id", "title", "globalId", "categoryId", "categoryName", "url", "location", "shippingType", "shippingLocations",
                "shippingTime", "startTime", "endTime", "returnsAccepted", "conditionId", "listingIsTopRated", 
               "sellerFeedbackScore", "sellerPositivePercent", "sellerName", "sellerIsTopRated", "price", "currency", "bids"]

## UK Clocks

In [3]:
uk_clocks = pd.DataFrame(QueryAll("clock_uk"), columns=cols)

In [4]:
uk_clocks = uk_clocks.drop(['currency', 'returnsAccepted','shippingTime'], axis=1)
#since all the values in these three colums are negative one, delete all three columns

In [5]:
uk_clocks = uk_clocks.drop(uk_clocks[uk_clocks['bids'] == -1].index, axis=0)
uk_clocks = uk_clocks.drop(uk_clocks[uk_clocks['bids'] == -2].index, axis=0)

In [6]:
uk_clocks['price'] = uk_clocks['price'].apply(convert_currency)

In [7]:
zscore = (uk_clocks.price - uk_clocks.price.mean() ) / uk_clocks.price.std()
uk_clocks = uk_clocks[zscore < 2.5]

In [8]:
uk_clocks['endAtWeekend'] = uk_clocks["endTime"].str.slice(0, 10).isin(WEEKENDS).astype(int)

In [9]:
uk_clocks['endAtEvening'] = uk_clocks["endTime"].str.slice(11, 13).astype(int).gt(16).astype(int)

In [10]:
uk_clocks['length'] = (pd.to_datetime(uk_clocks['endTime'])-pd.to_datetime(uk_clocks['startTime'])).dt.days

In [11]:
uk_clocks['isBroken'] = uk_clocks['conditionId'].isin(BROKEN).astype(int)
uk_clocks['isUsed'] = uk_clocks['conditionId'].isin(USED).astype(int)

In [12]:
uk_clocks['isLarge'] = uk_clocks['title'].str.lower().str.contains('large').astype(int)

In [13]:
uk_clocks['freeShipping'] = (uk_clocks['shippingType'] == 'Free').astype(int)

In [14]:
uk_clocks['isSold'] = (uk_clocks['bids'] > 0).astype(int)

In [15]:
uk_clocks = uk_clocks.drop(['title', 'url', 'globalId', 'categoryId', 'categoryName', 'location', 'shippingType',
                     'shippingLocations', 'startTime', 'endTime', 'conditionId', 'sellerName', 'bids'], axis=1)

In [16]:
uk_clocks = uk_clocks.reset_index(drop=True)
uk_clocks.to_csv('uk_clocks.csv', index=False)

## US Clocks

In [17]:
us_clocks = pd.DataFrame(QueryAll("clock_us"), columns=cols)

In [18]:
us_clocks = us_clocks.drop(['currency', 'returnsAccepted','shippingTime'], axis=1)
#since all the values in these three colums are negative one, delete all three columns

In [19]:
us_clocks = us_clocks.drop(us_clocks[us_clocks['bids'] == -1].index, axis=0)
us_clocks = us_clocks.drop(us_clocks[us_clocks['bids'] == -2].index, axis=0)

In [20]:
us_clocks['price'] = us_clocks['price'].apply(convert_currency)

In [21]:
zscore = (us_clocks.price - us_clocks.price.mean() ) / us_clocks.price.std()
us_clocks = us_clocks[zscore < 2.5]

In [22]:
us_clocks['endAtWeekend'] = us_clocks["endTime"].str.slice(0, 10).isin(WEEKENDS).astype(int)

In [23]:
us_clocks['endAtEvening'] = us_clocks["endTime"].str.slice(11, 13).astype(int).lt(8).astype(int)

In [24]:
us_clocks['length'] = (pd.to_datetime(us_clocks['endTime'])-pd.to_datetime(us_clocks['startTime'])).dt.days

In [25]:
us_clocks['isBroken'] = us_clocks['conditionId'].isin(BROKEN).astype(int)
us_clocks['isUsed'] = us_clocks['conditionId'].isin(USED).astype(int)

In [26]:
us_clocks['isLarge'] = us_clocks['title'].str.lower().str.contains('large').astype(int)

In [27]:
us_clocks['freeShipping'] = (us_clocks['shippingType'] == 'Free').astype(int)

In [28]:
us_clocks['isSold'] = (us_clocks['bids'] > 0).astype(int)

In [29]:
us_clocks = us_clocks.drop(['title', 'url', 'globalId', 'categoryId', 'categoryName', 'location', 'shippingType',
                     'shippingLocations', 'startTime', 'endTime', 'conditionId', 'sellerName', 'bids'], axis=1)

In [30]:
us_clocks = us_clocks.reset_index(drop=True)
us_clocks.to_csv('us_clocks.csv', index=False)

## UK Books

In [31]:
uk_books = pd.DataFrame(QueryAll("book_uk"), columns=cols)

In [32]:
uk_books = uk_books.drop(['currency', 'returnsAccepted','shippingTime'], axis=1)
#since all the values in these three colums are negative one, delete all three columns

In [33]:
uk_books = uk_books.drop(uk_books[uk_books['bids'] == -1].index, axis=0)
uk_books = uk_books.drop(uk_books[uk_books['bids'] == -2].index, axis=0)

In [34]:
uk_books['price'] = uk_books['price'].apply(convert_currency)

In [35]:
zscore = (uk_books.price - uk_books.price.mean() ) / uk_books.price.std()
uk_books = uk_books[zscore < 2.5]

In [36]:
uk_books['endAtWeekend'] = uk_books["endTime"].str.slice(0, 10).isin(WEEKENDS).astype(int)

In [37]:
uk_books['endAtEvening'] = uk_books["endTime"].str.slice(11, 13).astype(int).gt(16).astype(int)

In [38]:
uk_books['length'] = (pd.to_datetime(uk_books['endTime'])-pd.to_datetime(uk_books['startTime'])).dt.days

In [39]:
uk_books['isBroken'] = uk_books['conditionId'].isin(BROKEN).astype(int)
uk_books['isUsed'] = uk_books['conditionId'].isin(USED).astype(int)

In [40]:
uk_books['isSet'] = uk_books['title'].str.lower().str.contains('set').astype(int)

In [41]:
uk_books['freeShipping'] = (uk_books['shippingType'] == 'Free').astype(int)

In [42]:
uk_books['isSold'] = (uk_books['bids'] > 0).astype(int)

In [43]:
uk_books = uk_books.drop(['title', 'url', 'globalId', 'categoryId', 'categoryName', 'location', 'shippingType',
                     'shippingLocations', 'startTime', 'endTime', 'conditionId', 'sellerName', 'bids'], axis=1)

In [44]:
uk_books = uk_books.reset_index(drop=True)
uk_books.to_csv('uk_books.csv', index=False)

## US Books

In [45]:
us_books = pd.DataFrame(QueryAll("book_us"), columns=cols)

In [46]:
us_books = us_books.drop(['currency', 'returnsAccepted','shippingTime'], axis=1)
#since all the values in these three colums are negative one, delete all three columns

In [47]:
us_books = us_books.drop(us_books[us_books['bids'] == -1].index, axis=0)
us_books = us_books.drop(us_books[us_books['bids'] == -2].index, axis=0)

In [48]:
us_books['price'] = us_books['price'].apply(convert_currency)

In [49]:
zscore = (us_books.price - us_books.price.mean() ) / us_books.price.std()
us_books = us_books[zscore < 2.5]

In [50]:
us_books['endAtWeekend'] = us_books["endTime"].str.slice(0, 10).isin(WEEKENDS).astype(int)

In [51]:
us_books['endAtEvening'] = us_books["endTime"].str.slice(11, 13).astype(int).lt(8).astype(int)

In [52]:
us_books['length'] = (pd.to_datetime(us_books['endTime'])-pd.to_datetime(us_books['startTime'])).dt.days

In [53]:
us_books['isBroken'] = us_books['conditionId'].isin(BROKEN).astype(int)
us_books['isUsed'] = us_books['conditionId'].isin(USED).astype(int)

In [54]:
us_books['isSet'] = us_books['title'].str.lower().str.contains('set').astype(int)

In [55]:
us_books['freeShipping'] = (us_books['shippingType'] == 'Free').astype(int)

In [56]:
us_books['isSold'] = (us_books['bids'] > 0).astype(int)

In [57]:
us_books = us_books.drop(['title', 'url', 'globalId', 'categoryId', 'categoryName', 'location', 'shippingType',
                     'shippingLocations', 'startTime', 'endTime', 'conditionId', 'sellerName', 'bids'], axis=1)

In [58]:
us_books = us_books.reset_index(drop=True)
us_books.to_csv('us_books.csv', index=False)